<a href="https://colab.research.google.com/github/mynameislllyt/API_Experiment/blob/main/Biyue_API_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip -q "/content/drive/MyDrive/API_Detection.zip" -d /content
%cd /content/API_Detection


/content/API_Detection


In [3]:
!python -m pip install -U pip
!grep -vE "^(torch|torchvision|torchaudio)\b" requirements.txt > requirements_notorch.txt
!pip install -r requirements_notorch.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 35.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [4]:
!python -c "import torch; print(torch.__version__); print('cuda available:', torch.cuda.is_available()); print('gpu:', torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)"


2.9.0+cu126
cuda available: True
gpu: Tesla T4


In [5]:
!python src/train.py --data_path data/dynamic_api_call_sequence_per_malware_100_0_306.csv


使用设备: cuda

步骤 1: 加载数据
数据集统计:
  样本数量: 43876
  序列长度: 100
  API词汇表大小: 307
  正常样本数: 1079
  异常样本数: 42797

步骤 2: 数据预处理

数据集划分:
  正常样本: 1079
  异常样本: 42797
滑动窗口生成:
  原始样本数: 863
  窗口大小: 10
  生成窗口数: 78533
滑动窗口生成:
  原始样本数: 216
  窗口大小: 10
  生成窗口数: 19656
滑动窗口生成:
  原始样本数: 42797
  窗口大小: 10
  生成窗口数: 3894527

最终数据集:
  训练窗口数: 78533 (全部为正常样本)
  测试窗口数: 3914183
    - 正常: 19656
    - 异常: 3894527

步骤 3: 创建TE-LSTM模型
模型参数:
  词汇表大小: 307
  模型维度: 128
  注意力头数: 8
  Encoder层数: 2
  LSTM隐藏维度: 64
  总参数量: 566,707
Traceback (most recent call last):
  File "/content/API_Detection/src/train.py", line 365, in <module>
    main(args)
  File "/content/API_Detection/src/train.py", line 211, in main
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'


In [8]:
!grep -n "ReduceLROnPlateau" -n src/train.py
!sed -i -E "s/, *verbose *= *[A-Za-z]+//g; s/verbose *= *[A-Za-z]+, *//g" src/train.py
!grep -n "ReduceLROnPlateau" -n src/train.py
!python src/train.py --data_path data/dynamic_api_call_sequence_per_malware_100_0_306.csv --batch_size 256 --epochs 20



211:    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
211:    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
使用设备: cuda

步骤 1: 加载数据
数据集统计:
  样本数量: 43876
  序列长度: 100
  API词汇表大小: 307
  正常样本数: 1079
  异常样本数: 42797

步骤 2: 数据预处理

数据集划分:
  正常样本: 1079
  异常样本: 42797
滑动窗口生成:
  原始样本数: 863
  窗口大小: 10
  生成窗口数: 78533
滑动窗口生成:
  原始样本数: 216
  窗口大小: 10
  生成窗口数: 19656
滑动窗口生成:
  原始样本数: 42797
  窗口大小: 10
  生成窗口数: 3894527

最终数据集:
  训练窗口数: 78533 (全部为正常样本)
  测试窗口数: 3914183
    - 正常: 19656
    - 异常: 3894527

步骤 3: 创建TE-LSTM模型
模型参数:
  词汇表大小: 307
  模型维度: 128
  注意力头数: 8
  Encoder层数: 2
  LSTM隐藏维度: 64
  总参数量: 566,707

步骤 4: 训练模型

Epoch 1/20
--------------------------------------------------
Training: 100% 306/306 [00:03<00:00, 95.66it/s, loss=0.0549] 
Train Loss: 1.0867 | Val Loss: 0.1205
✓ 保存最佳模型 (Val Loss: 0.1205)

Epoch 2/20
--------------------------------------------------
Training: 100% 306/306 [00:02<00:00, 117.57it/s, loss=0.0138]
Train Loss: 0.0221 | Val Loss: 0.0580
✓ 保存最佳模型 (Val Loss: 0.0580)

Epoc

In [9]:
!python src/evaluate.py -h



usage: evaluate.py [-h] --model_path MODEL_PATH --data_path DATA_PATH
                   [--batch_size BATCH_SIZE]
                   [--threshold_percentile THRESHOLD_PERCENTILE]
                   [--result_dir RESULT_DIR]

评估TE-LSTM异常检测模型

options:
  -h, --help            show this help message and exit
  --model_path MODEL_PATH
                        模型文件路径
  --data_path DATA_PATH
                        数据集路径
  --batch_size BATCH_SIZE
                        批次大小 (默认: 32)
  --threshold_percentile THRESHOLD_PERCENTILE
                        异常阈值百分位数 (默认: 95)
  --result_dir RESULT_DIR
                        结果保存目录 (默认: results)


In [10]:
!find . -maxdepth 3 -type f \( -name "*.pt" -o -name "*.pth" -o -name "*.ckpt" \)


./models/te_lstm_best.pth
./models/te_lstm_final.pth


In [11]:
!python src/evaluate.py \
  --model_path models/te_lstm_best.pth \
  --data_path data/dynamic_api_call_sequence_per_malware_100_0_306.csv \
  --batch_size 256 \
  --threshold_percentile 95 \
  --result_dir results


使用设备: cuda

步骤 1: 加载模型
加载模型: models/te_lstm_best.pth
训练轮次: 10
训练损失: 0.0008
验证损失: 0.0169

步骤 2: 加载测试数据
数据集统计:
  样本数量: 43876
  序列长度: 100
  API词汇表大小: 307
  正常样本数: 1079
  异常样本数: 42797

数据集划分:
  正常样本: 1079
  异常样本: 42797
滑动窗口生成:
  原始样本数: 863
  窗口大小: 10
  生成窗口数: 78533
滑动窗口生成:
  原始样本数: 216
  窗口大小: 10
  生成窗口数: 19656
滑动窗口生成:
  原始样本数: 42797
  窗口大小: 10
  生成窗口数: 3894527

最终数据集:
  训练窗口数: 78533 (全部为正常样本)
  测试窗口数: 3914183
    - 正常: 19656
    - 异常: 3894527

步骤 3: 配置异常检测器
从文件加载阈值: 0.0004
异常检测阈值设定为: 0.0004 (第95百分位)

步骤 4: 评估模型
正在评估模型...

评估结果

准确率 (Accuracy):  0.1252
精确率 (Precision): 0.9977
召回率 (Recall):    0.1211
F1分数 (F1-Score):  0.2160
AUC-ROC:            0.5927

混淆矩阵:
                预测正常  预测异常
实际正常       18573      1083
实际异常     3422857    471670

误差统计:
正常样本平均误差: 0.0016 (±0.0336)
异常样本平均误差: 0.0170 (±0.1686)

详细分类报告:
              precision    recall  f1-score   support

      Normal       0.01      0.94      0.01     19656
     Anomaly       1.00      0.12      0.22   3894527

    accuracy           